# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [15]:
# Instalamos el modelo predictivo Prophet
!python -m pip install prophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [16]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [17]:
today = '2023-06-21'# y fecha de fin (hoy) en formato texto '%Y-%m-%d'
start_date = '2016-01-01'# Fecha de inicio es un string 2016-01-01
# Descargar el dataframe
btc_df = yf.download('BTC=F',start_date, today).reset_index()

btc_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
1381,2023-06-13,"$26,010.00","$26,555.00","$25,725.00","$25,930.00","$25,930.00",5185
1382,2023-06-14,"$25,890.00","$26,155.00","$24,815.00","$25,900.00","$25,900.00",5866
1383,2023-06-15,"$25,110.00","$25,620.00","$24,745.00","$25,470.00","$25,470.00",10605
1384,2023-06-16,"$25,600.00","$26,600.00","$25,145.00","$26,445.00","$26,445.00",10238
1385,2023-06-20,"$26,485.00","$28,445.00","$26,310.00","$28,180.00","$28,180.00",10238


In [35]:
# Validamos que no hay datos vacíos
btc_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [27]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = btc_df[["Date", "Open"]]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds",
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df = df.rename(columns =new_names)

In [28]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [29]:
df.tail()

,ds,y
1381,2023-06-13,"$26,010.00"
1382,2023-06-14,"$25,890.00"
1383,2023-06-15,"$25,110.00"
1384,2023-06-16,"$25,600.00"
1385,2023-06-20,"$26,485.00"


In [36]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = ""
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [38]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet(seasonality_mode= "multiplicative")

In [39]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpu8lou95i/rj6nxgi_.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpu8lou95i/a6cflcey.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=48014', 'data', 'file=/tmp/tmpu8lou95i/rj6nxgi_.json', 'init=/tmp/tmpu8lou95i/a6cflcey.json', 'output', 'file=/tmp/tmpu8lou95i/prophet_model66g105rn/prophet_model-20230621094244.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
09:42:44 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:42:48 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [41]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(365)
future.tail()

,ds
1746,2024-06-15
1747,2024-06-16
1748,2024-06-17
1749,2024-06-18
1750,2024-06-19


In [42]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2017-12-18
1,2017-12-19
2,2017-12-20
3,2017-12-21
4,2017-12-22
...,...
1746,2024-06-15
1747,2024-06-16
1748,2024-06-17
1749,2024-06-18


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [43]:
#Utiliza el modelo m para predecir el dataframe future
forecast =m.predict(future)

In [44]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1746,2024-06-15,"$-35,643.08","$-211,434.31","$131,380.39"
1747,2024-06-16,"$-35,698.00","$-213,244.05","$132,439.37"
1748,2024-06-17,"$20,217.42","$-75,681.46","$122,480.75"
1749,2024-06-18,"$20,155.66","$-74,405.24","$121,937.60"
1750,2024-06-19,"$20,023.87","$-75,990.85","$120,070.80"


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [45]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)